In [16]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [18]:
df = pd.DataFrame()

for i in range(1, 13):
    if i == 1:
        test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 100000;"
        df = create_pandas_table(test2)
    else:
        test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 100000;"
        test_pd_table = create_pandas_table(test2)
        df = pd.concat([df, test_pd_table]) 

In [19]:
passengers = create_pandas_table("SELECT * FROM passengers TABLESAMPLE SYSTEM(10) WHERE year >= 2019 ")

In [20]:
passengers.shape

(47866, 38)

In [21]:
passengers.passengers.max()

68773.0

In [22]:
fuel = create_pandas_table("SELECT * FROM fuel_comsumption WHERE year >= 2019 ")

In [23]:
fuel.shape
fuel.total_gallons

0        1235210.0
1              0.0
2          60821.0
3              0.0
4         410186.0
          ...     
453      3943414.0
454            0.0
455            0.0
456        22895.0
457    162505172.0
Name: total_gallons, Length: 458, dtype: float64

In [24]:
fuel_mean=fuel[['unique_carrier','total_gallons']].groupby(['unique_carrier'],as_index=False).mean()

In [25]:
fuel_mean.rename(columns={'unique_carrier':'op_unique_carrier'},inplace=True)


In [26]:
passengers['capacity'] = passengers['seats'] - passengers['passengers']


In [27]:
passengers.rename(columns={'unique_carrier':'mkt_unique_carrier'},inplace=True)


In [28]:
passengers_mean=passengers[['mkt_unique_carrier','origin','dest','passengers', 'capacity']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [29]:
passengers_mean.head()

,mkt_unique_carrier,origin,dest,passengers,capacity
0,04Q,ACK,BDR,19.0,5.000000
1,04Q,ACK,HPN,287.0,139.666667
2,04Q,ACK,TEB,1.0,7.000000
3,04Q,AGC,BED,1.0,7.000000
4,04Q,ALB,BTV,2.0,6.000000


In [30]:
df_merged=pd.merge(df,passengers_mean,on=['mkt_unique_carrier', 'origin','dest'], how='inner') 

In [31]:
df_merged=pd.merge(df_merged,fuel_mean, on=['op_unique_carrier'], how='inner') 

In [32]:
df_merged.shape

(790826, 45)

In [33]:
df_merged.isna().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
dep_delay                   0
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
cancellation_code      790826
diverted                    0
dup                         0
crs_elapsed_time            0
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_de

In [34]:
# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()

In [35]:
df_merged.passengers.unique()

array([5038.6,  124. , 2297. , ...,  206. , 6696. ,  461. ])

In [36]:
df.mkt_unique_carrier.unique()

array(['AA', 'NK', 'AS', 'DL', 'UA', 'WN', 'G4', 'HA', 'B6', 'F9'],
      dtype=object)

In [37]:
df.shape

(1179349, 42)

In [38]:
df_merged.shape

(790826, 45)

In [39]:
df_merged.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,passengers,capacity,total_gallons
0,2019-01-01,NK,NK,NK,811,NK,N640NK,811,13204,MCO,...,14.0,0.0,0.0,None,NaN,NaN,None,5038.6,600.6,3.937194e+07
1,2019-01-01,NK,NK,NK,803,NK,N625NK,803,13204,MCO,...,1.0,0.0,66.0,None,NaN,NaN,None,5038.6,600.6,3.937194e+07
2,2019-01-02,NK,NK,NK,811,NK,N646NK,811,13204,MCO,...,15.0,0.0,0.0,None,NaN,NaN,None,5038.6,600.6,3.937194e+07
3,2019-01-03,NK,NK,NK,803,NK,N605NK,803,13204,MCO,...,1.0,0.0,39.0,None,NaN,NaN,None,5038.6,600.6,3.937194e+07
4,2019-01-04,NK,NK,NK,811,NK,N635NK,811,13204,MCO,...,11.0,0.0,0.0,None,NaN,NaN,None,5038.6,600.6,3.937194e+07


In [40]:
df_test = df_merged[['mkt_unique_carrier','origin','dest','dep_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'weather_delay', 'carrier_delay']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [41]:
df_merged.drop(columns=['first_dep_time','total_add_gtime','longest_add_gtime', 'no_name', 'dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay'], inplace=True)

In [42]:
df_merged = pd.merge(df_merged, df_test, on=['mkt_unique_carrier', 'origin', 'dest'] )

KeyError: "['first_dep_time' 'total_add_gtime' 'longest_add_gtime' 'no_name'] not found in axis"

In [69]:
df_merged = df_merged.drop(columns=['dep_delay_x', 'carrier_delay_x', 'weather_delay_x', 'nas_delay_x', 'security_delay_x',
       'late_aircraft_delay_x',])

In [74]:
len(df_merged.columns)

41

In [ ]:
df_merged.rename(columns={"capacity": "a", "B": "b", "C": "c"}

In [44]:
df_merged

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay,nas_delay,security_delay,late_aircraft_delay,weather_delay,carrier_delay
0,2019-01-01,NK,NK,NK,811,NK,N640NK,811,13204,MCO,...,1189.0,5038.6,600.6,3.937194e+07,48.016854,20.174157,0.101124,16.831461,3.0,8.213483
1,2019-01-01,NK,NK,NK,803,NK,N625NK,803,13204,MCO,...,1189.0,5038.6,600.6,3.937194e+07,48.016854,20.174157,0.101124,16.831461,3.0,8.213483
2,2019-01-02,NK,NK,NK,811,NK,N646NK,811,13204,MCO,...,1189.0,5038.6,600.6,3.937194e+07,48.016854,20.174157,0.101124,16.831461,3.0,8.213483
3,2019-01-03,NK,NK,NK,803,NK,N605NK,803,13204,MCO,...,1189.0,5038.6,600.6,3.937194e+07,48.016854,20.174157,0.101124,16.831461,3.0,8.213483
4,2019-01-04,NK,NK,NK,811,NK,N635NK,811,13204,MCO,...,1189.0,5038.6,600.6,3.937194e+07,48.016854,20.174157,0.101124,16.831461,3.0,8.213483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790821,2019-12-13,F9,F9,F9,2938,F9,N332FR,2938,13303,MIA,...,1258.0,491.0,49.0,2.353042e+07,73.750000,43.500000,0.000000,15.000000,0.0,19.500000
790822,2019-12-14,F9,F9,F9,2938,F9,N313FR,2938,13303,MIA,...,1258.0,491.0,49.0,2.353042e+07,73.750000,43.500000,0.000000,15.000000,0.0,19.500000
790823,2019-12-21,F9,F9,F9,2938,F9,N230FR,2938,13303,MIA,...,1258.0,491.0,49.0,2.353042e+07,73.750000,43.500000,0.000000,15.000000,0.0,19.500000
790824,2019-12-09,F9,F9,F9,2167,F9,N339FR,2167,12889,LAS,...,867.0,151.0,35.0,2.353042e+07,12.500000,12.500000,0.000000,0.000000,0.0,17.500000


In [45]:
df_merged.to_csv('flights_merged.csv')